In [153]:
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.patheffects as mpe
import matplotlib
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics


In [154]:
df = pd.read_csv("data_all_labels.csv")

In [155]:
def Classification(CLASS):
    if CLASS >= 0 and CLASS <= 2:
        return 'Poor'
    elif CLASS > 2 and CLASS <= 3:
        return 'Average'
    elif CLASS >3 and CLASS <= 5:
        return 'Good'

In [156]:
df['n_class']= df['P1203'].apply(Classification)

In [157]:
df['Case'].unique()

array(['Driving-1', 'Static-1', 'Driving-2', 'Driving-3'], dtype=object)

In [158]:
df.columns

Index(['sr#', 'ID', 'ExpID', 'Case', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'AveragePTime', 'TP', 'TPT', 'Total_Packets',
       'AveragePTime_gt100', 'InterIntervalTime_gt100', 'Total_Packets_gt100',
       'Width_Height', 'P1203', 'Stall', 'Bitrate', 'P1203_2C', 'Stall_Binary',
       'n_class'],
      dtype='object')

In [159]:
df['n_class']

0           Poor
1           Poor
2           Poor
3        Average
4        Average
          ...   
15207    Average
15208    Average
15209       Poor
15210       Poor
15211       Poor
Name: n_class, Length: 15212, dtype: object

In [160]:
# df =df[(df['Case']=='Static-1')]

In [161]:
df =df[(df['Case']=='Driving-2') | (df['Case']=='Driving-3')]

In [162]:
df =df[(df['Algo']=='bba')| (df['Algo']=='elastic')| (df['Algo']=='conventional')]

In [163]:
df.shape[0]

7858

In [164]:
df['n_class'].unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [165]:
df.isnull().values.any()

True

In [166]:
nan_cols = [i for i in df.columns if df[i].isnull().any()]

In [167]:
nan_cols

['sr#']

In [168]:
# df['Min_gt100']= df['Min_gt100'].fillna(value=df['Min_gt100'].mean())

In [169]:
# df['Max_gt100']= df['Max_gt100'].fillna(value=df['Max_gt100'].mean())

In [170]:
# df.loc[:, (df == 0).all()]


In [171]:
from sklearn import preprocessing


In [172]:
X = df[['TPT', 'Total_Packets_gt100','Total_Packets',
        'InterIntervalTime_gt100','AveragePTime_gt100','AveragePTime','InterIntervalTime']]
y = df['n_class']

In [173]:
X.columns

Index(['TPT', 'Total_Packets_gt100', 'Total_Packets',
       'InterIntervalTime_gt100', 'AveragePTime_gt100', 'AveragePTime',
       'InterIntervalTime'],
      dtype='object')

In [174]:
# X=preprocessing.normalize(X)
X= preprocessing.scale(X)

In [175]:
# y = df.iloc[:, 23]

In [176]:
y

3301        Poor
3302        Poor
3303        Poor
3304     Average
3305     Average
          ...   
15037       Good
15038       Good
15039       Good
15040       Good
15041       Good
Name: n_class, Length: 7858, dtype: object

In [177]:
y.unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [178]:
X.shape[0]

7858

In [179]:
sm = SMOTE()
X, y = sm.fit_sample(X, y)

In [180]:
X.shape[0]

10896

In [181]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [182]:
dummy_y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [183]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=7,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(150,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(250,kernel_initializer = 'uniform',  activation='relu'))

    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [184]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=128, verbose=1)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/500
69/69 [==============================] - 0s 1ms/step - loss: 0.8484 - accuracy: 0.5911
Epoch 2/500
69/69 [==============================] - 0s 1ms/step - loss: 0.6529 - accuracy: 0.6833
Epoch 3/500
69/69 [==============================] - 0s 1ms/step - loss: 0.6213 - accuracy: 0.7058
Epoch 4/500
69/69 [==============================] - 0s 1ms/step - loss: 0.6031 - accuracy: 0.7196
Epoch 5/500
69/69 [==============================] - 0s 2ms/step - loss: 0.5910 - accuracy: 0.7251
Epoch 6/500
69/69 [==============================] - 0s 1ms/step - loss: 0.5806 - accuracy: 0.7345
Epoch 7/500
69/69 [==============================] - 0s 2ms/step - loss: 0.5842 - accuracy: 0.7250
Epoch 8/500
69/69 [==============================] - 0s 1ms/step - loss: 0.5734 - accuracy: 0.7315
Epoch 9/500
69/69 [==============================] - 0s 1ms/step - loss: 0.5692 - accuracy: 0.7362
Epoch 10/500
69/69 [==============================] - 0s 2ms/step - loss: 0.5561 - accuracy: 0.7380
Epoch 11/

69/69 [==============================] - 0s 3ms/step - loss: 0.4520 - accuracy: 0.7848
Epoch 84/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4408 - accuracy: 0.7919
Epoch 85/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4407 - accuracy: 0.7900
Epoch 86/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.7882
Epoch 87/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4356 - accuracy: 0.7904
Epoch 88/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4434 - accuracy: 0.7920
Epoch 89/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4388 - accuracy: 0.7890
Epoch 90/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4411 - accuracy: 0.7923
Epoch 91/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.7920
Epoch 92/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4335 - accuracy: 0.7934
Epoch 93/500


69/69 [==============================] - 0s 2ms/step - loss: 0.3828 - accuracy: 0.8211
Epoch 165/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3870 - accuracy: 0.8176
Epoch 166/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4000 - accuracy: 0.8116
Epoch 167/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3865 - accuracy: 0.8171
Epoch 168/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3928 - accuracy: 0.8157
Epoch 169/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3819 - accuracy: 0.8193
Epoch 170/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3816 - accuracy: 0.8176
Epoch 171/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3830 - accuracy: 0.8241
Epoch 172/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3856 - accuracy: 0.8224
Epoch 173/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8129
Epoc

69/69 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.8327
Epoch 246/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3554 - accuracy: 0.8313
Epoch 247/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3500 - accuracy: 0.8347
Epoch 248/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3558 - accuracy: 0.8338
Epoch 249/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3641 - accuracy: 0.8293
Epoch 250/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3565 - accuracy: 0.8331
Epoch 251/500
69/69 [==============================] - 0s 3ms/step - loss: 0.3462 - accuracy: 0.8410
Epoch 252/500
69/69 [==============================] - 0s 3ms/step - loss: 0.3535 - accuracy: 0.8351
Epoch 253/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3466 - accuracy: 0.8351
Epoch 254/500
69/69 [==============================] - 0s 3ms/step - loss: 0.3444 - accuracy: 0.8388
Epoc

69/69 [==============================] - 0s 2ms/step - loss: 0.3261 - accuracy: 0.8468
Epoch 327/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3253 - accuracy: 0.8488
Epoch 328/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8417
Epoch 329/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3281 - accuracy: 0.8437
Epoch 330/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3203 - accuracy: 0.8508
Epoch 331/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3203 - accuracy: 0.8495
Epoch 332/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3203 - accuracy: 0.8497
Epoch 333/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3230 - accuracy: 0.8489
Epoch 334/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3274 - accuracy: 0.8465
Epoch 335/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3194 - accuracy: 0.8490
Epoc

...

Epoch 107/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4313 - accuracy: 0.7917
Epoch 108/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4379 - accuracy: 0.7886
Epoch 109/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4315 - accuracy: 0.7944
Epoch 110/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4305 - accuracy: 0.7987
Epoch 111/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4432 - accuracy: 0.7865
Epoch 112/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4248 - accuracy: 0.7956
Epoch 113/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4268 - accuracy: 0.7951
Epoch 114/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4249 - accuracy: 0.7979
Epoch 115/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.7990
Epoch 116/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4266 - accurac

69/69 [==============================] - 0s 2ms/step - loss: 0.3830 - accuracy: 0.8182
Epoch 189/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3812 - accuracy: 0.8222
Epoch 190/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3835 - accuracy: 0.8139
Epoch 191/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3757 - accuracy: 0.8263
Epoch 192/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3762 - accuracy: 0.8241
Epoch 193/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.8223
Epoch 194/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3728 - accuracy: 0.8255
Epoch 195/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3770 - accuracy: 0.8268
Epoch 196/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3747 - accuracy: 0.8218
Epoch 197/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3748 - accuracy: 0.8275
Epoc

69/69 [==============================] - 0s 2ms/step - loss: 0.3369 - accuracy: 0.8430
Epoch 270/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3481 - accuracy: 0.8384
Epoch 271/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3490 - accuracy: 0.8353
Epoch 272/500
69/69 [==============================] - 0s 3ms/step - loss: 0.3357 - accuracy: 0.8440
Epoch 273/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3345 - accuracy: 0.8460
Epoch 274/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3430 - accuracy: 0.8391
Epoch 275/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3456 - accuracy: 0.8401
Epoch 276/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3528 - accuracy: 0.8337
Epoch 277/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3398 - accuracy: 0.8409
Epoch 278/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3384 - accuracy: 0.8412
Epoc

69/69 [==============================] - 0s 2ms/step - loss: 0.3166 - accuracy: 0.8519
Epoch 351/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3181 - accuracy: 0.8470
Epoch 352/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3141 - accuracy: 0.8514
Epoch 353/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3162 - accuracy: 0.8540
Epoch 354/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3230 - accuracy: 0.8491
Epoch 355/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3226 - accuracy: 0.8481
Epoch 356/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3147 - accuracy: 0.8544
Epoch 357/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3103 - accuracy: 0.8541
Epoch 358/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3131 - accuracy: 0.8545
Epoch 359/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3128 - accuracy: 0.8537
Epoc

69/69 [==============================] - 0s 2ms/step - loss: 0.3137 - accuracy: 0.8561
Epoch 432/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3143 - accuracy: 0.8565
Epoch 433/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2960 - accuracy: 0.8636
Epoch 434/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3316 - accuracy: 0.8533
Epoch 435/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3002 - accuracy: 0.8588
Epoch 436/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3025 - accuracy: 0.8591
Epoch 437/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2976 - accuracy: 0.8612
Epoch 438/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2966 - accuracy: 0.8635
Epoch 439/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2903 - accuracy: 0.8645
Epoch 440/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2897 - accuracy: 0.8628
Epoc

In [185]:
print(results)

[0.82201833 0.83570445 0.82606703 0.82514918 0.8155117 ]


In [186]:
# estimator.fit(X_train, y_train)
# predictions = estimator.predict(X_test)
#print(predictions)

In [187]:
# print(encoder.inverse_transform(predictions))

In [188]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

Epoch 1/500
60/60 [==============================] - 0s 2ms/step - loss: 0.8698 - accuracy: 0.6157
Epoch 2/500
60/60 [==============================] - 0s 2ms/step - loss: 0.6680 - accuracy: 0.6815
Epoch 3/500
60/60 [==============================] - 0s 2ms/step - loss: 0.6265 - accuracy: 0.7064
Epoch 4/500
60/60 [==============================] - 0s 2ms/step - loss: 0.6103 - accuracy: 0.7152
Epoch 5/500
60/60 [==============================] - 0s 2ms/step - loss: 0.6005 - accuracy: 0.7230
Epoch 6/500
60/60 [==============================] - 0s 2ms/step - loss: 0.5928 - accuracy: 0.7252
Epoch 7/500
60/60 [==============================] - 0s 2ms/step - loss: 0.5847 - accuracy: 0.7304
Epoch 8/500
60/60 [==============================] - 0s 2ms/step - loss: 0.5894 - accuracy: 0.7235
Epoch 9/500
60/60 [==============================] - 0s 2ms/step - loss: 0.5796 - accuracy: 0.7243
Epoch 10/500
60/60 [==============================] - 0s 2ms/step - loss: 0.5708 - accuracy: 0.7334
Epoch 11/

60/60 [==============================] - 0s 2ms/step - loss: 0.4552 - accuracy: 0.7822
Epoch 84/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.7850
Epoch 85/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4507 - accuracy: 0.7869
Epoch 86/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.7869
Epoch 87/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4526 - accuracy: 0.7851
Epoch 88/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4517 - accuracy: 0.7859
Epoch 89/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4428 - accuracy: 0.7889
Epoch 90/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4500 - accuracy: 0.7905
Epoch 91/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4523 - accuracy: 0.7844
Epoch 92/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4526 - accuracy: 0.7799
Epoch 93/500


60/60 [==============================] - 0s 2ms/step - loss: 0.3973 - accuracy: 0.8175
Epoch 165/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3996 - accuracy: 0.8095
Epoch 166/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4090 - accuracy: 0.8054
Epoch 167/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4062 - accuracy: 0.8078
Epoch 168/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4016 - accuracy: 0.8061
Epoch 169/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3983 - accuracy: 0.8101
Epoch 170/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3929 - accuracy: 0.8175
Epoch 171/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4011 - accuracy: 0.8062
Epoch 172/500
60/60 [==============================] - 0s 2ms/step - loss: 0.4017 - accuracy: 0.8062
Epoch 173/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3925 - accuracy: 0.8136
Epoc

60/60 [==============================] - 0s 2ms/step - loss: 0.3576 - accuracy: 0.8361
Epoch 246/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3560 - accuracy: 0.8370
Epoch 247/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3607 - accuracy: 0.8336
Epoch 248/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3516 - accuracy: 0.8355
Epoch 249/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3600 - accuracy: 0.8323
Epoch 250/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3632 - accuracy: 0.8280
Epoch 251/500
60/60 [==============================] - 0s 3ms/step - loss: 0.3573 - accuracy: 0.8340
Epoch 252/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3634 - accuracy: 0.8318
Epoch 253/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3528 - accuracy: 0.8348
Epoch 254/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3530 - accuracy: 0.8356
Epoc

60/60 [==============================] - 0s 2ms/step - loss: 0.3413 - accuracy: 0.8455
Epoch 327/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3230 - accuracy: 0.8490
Epoch 328/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3245 - accuracy: 0.8507
Epoch 329/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3268 - accuracy: 0.8503
Epoch 330/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3257 - accuracy: 0.8503
Epoch 331/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3278 - accuracy: 0.8500
Epoch 332/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3286 - accuracy: 0.8467
Epoch 333/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3261 - accuracy: 0.8504
Epoch 334/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3279 - accuracy: 0.8463
Epoch 335/500
60/60 [==============================] - 0s 3ms/step - loss: 0.3273 - accuracy: 0.8499
Epoc

60/60 [==============================] - 0s 2ms/step - loss: 0.3167 - accuracy: 0.8520
Epoch 408/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3099 - accuracy: 0.8563
Epoch 409/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3013 - accuracy: 0.8617
Epoch 410/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3037 - accuracy: 0.8596
Epoch 411/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3041 - accuracy: 0.8556
Epoch 412/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3027 - accuracy: 0.8584
Epoch 413/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2999 - accuracy: 0.8584
Epoch 414/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3079 - accuracy: 0.8535
Epoch 415/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3087 - accuracy: 0.8591
Epoch 416/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3099 - accuracy: 0.8581
Epoc

60/60 [==============================] - 0s 2ms/step - loss: 0.2943 - accuracy: 0.8659
Epoch 489/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2910 - accuracy: 0.8661
Epoch 490/500
60/60 [==============================] - 0s 2ms/step - loss: 0.3033 - accuracy: 0.8583
Epoch 491/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2813 - accuracy: 0.8716
Epoch 492/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2864 - accuracy: 0.8702
Epoch 493/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2931 - accuracy: 0.8659
Epoch 494/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2915 - accuracy: 0.8630
Epoch 495/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2793 - accuracy: 0.8715
Epoch 496/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2787 - accuracy: 0.8712
Epoch 497/500
60/60 [==============================] - 0s 2ms/step - loss: 0.2806 - accuracy: 0.8709
Epoc

In [189]:
print(predictions[0])
print(encoder.inverse_transform(predictions))

1
['Good' 'Average' 'Good' ... 'Good' 'Poor' 'Average']


In [190]:
Y_test[0]

array([0., 1., 0.], dtype=float32)

In [191]:
import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[3]
# 2

2

In [192]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, predictions)
cm

array([[995,  49,  74],
       [ 89, 924,  77],
       [136, 146, 779]])

In [193]:
print(classification_report(rounded_labels, predictions))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1118
           1       0.83      0.85      0.84      1090
           2       0.84      0.73      0.78      1061

    accuracy                           0.83      3269
   macro avg       0.83      0.82      0.82      3269
weighted avg       0.83      0.83      0.82      3269



In [194]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
accuracy_score(rounded_labels, predictions)
print('Accuracy is ' + str(round(accuracy_score(rounded_labels, predictions),2)))

Accuracy is 0.83


In [195]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [196]:
cmn= np.around(cm / cm.astype(np.float).sum(axis=1, keepdims=True),2) 

In [197]:
cmn

array([[0.89, 0.04, 0.07],
       [0.08, 0.85, 0.07],
       [0.13, 0.14, 0.73]])